# Задача  
**На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem**  
**Распишите, какие метрики и как вы будете анализировать**

# Гипотеза № 1

### **Гипотеза**  
Если на главной странице маркетплейса https://www.ozon.ru/ сделать баннер только под рекламу сторонних компаний, то количество переходов на сторонние сайты увеличиться на 10%. Увеличиться доход от рекламы. Но при этом показатели ARPU не уменьшаться т.к. не было внесенно существенных изменний по дизайну и структуре сайта.

### **Что делаем:**  
_Контрольная версия:_ Запускаем все в исходном виде.  
_Тестовая версия:_ Банер на главной странице предназначеный под рекламу. Запускаем только для рекламы сторонних компаний.

### **На каких пользователях тестируем:**  
На 5% от всех пользователей.

In [1]:
import plotly.graph_objects as go # Для построения графиков
from scipy import stats # Для расчёта ститистик
import numpy as np # Для работы с массивами данных
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm

import scipy.stats as st
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats import proportion
from statsmodels.stats.meta_analysis import effectsize_smd
import plotly.graph_objects as go
from typing import Union

In [2]:
# Вводим исходные данные по выборка
sample_1_click_control, sample_2_click_test = 15_000_000, (15_000_000 + (15_000_000*0.1)) * 0.05
sample_1_size_control, sample_2_size_test = 340_000_000, 340_000_000 * 0.05
prob_1 = sample_1_click_control / sample_1_size_control
prob_2 = sample_2_click_test / sample_2_size_test

# Считаем статистику по выборкам
se = np.sqrt((prob_1 * (1 - prob_1) / sample_1_size_control) + (prob_2 * (1 - prob_2) / sample_2_size_test))
z_score = (prob_1 - prob_2) / se
pvalue = stats.norm.sf(abs(z_score)) * 2

md = prob_1 - prob_2

print(f"Difference between samples: {md}, standart error: {se}, p_value: {pvalue}")

if pvalue < .05:
    print(f"Sample # {1 if md > 0 else 2} mean is greater")
else:
    print("No significant difference")

# Считаем доверительный интервал
left = md - 1.96 * se
right = md + 1.96 * se

left, right

Difference between samples: -0.004411764705882351, standart error: 5.329323540139159e-05, p_value: 0.0
Sample # 2 mean is greater


(-0.004516219447269079, -0.004307309964495624)

In [8]:
booted_diff = []
booted_pvalue = []
size = max(sample_1_click_control, sample_2_click_test)
for _ in tqdm(range(1000)):
    a_s = st.bernoulli.rvs(p=prob_1, size=size)
    b_s = st.bernoulli.rvs(p=prob_2, size=size)
    booted_diff.append(np.mean(a_s - b_s))
    booted_pvalue.append(proportion.proportions_chisquare([a_s.sum(), b_s.sum()], [a_s.size, b_s.size])[1])

md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
p_value_ci = 2 * (1 - st.norm.cdf(np.abs(md_ci / std_ci)))

100%|██████████| 1000/1000 [11:12<00:00,  1.49it/s]


In [4]:
# Доверительный интервал разниц между группами
alpha = 0.05
print(f"Mean difference: {np.round(md_ci, 5)}, p-value: {p_value_ci} and confidence interval is: [{np.round(left_ci, 5)}, {np.round(right_ci, 5)}]")

if p_value_ci < alpha and not min(left_ci, right_ci) < 0 < max(left_ci, right_ci):
    print(f"Sample # {1 if md_ci > 0 else 2} mean is greater")
else:
    print("No significant difference")

Mean difference: -0.00441, p-value: 0.0 and confidence interval is: [-0.00458, -0.00426]
Sample # 2 mean is greater


In [5]:
#Расчёт effect_size для пропорций
sample_1_click_control, sample_2_click_test = 15_000_000, (15_000_000 + (15_000_000*0.1)) * 0.05
sample_1_size_control, sample_2_size_test = 340_000_000, 340_000_000 * 0.05
prob_1, prob_2 = sample_1_click_control/sample_1_size_control, sample_2_click_test/sample_2_size_test
prob_1, prob_2
# es_formula = 2  * asin(np.sqrt(prob_1)) - 2 * asin(np.sqrt(prob_2))
# print(es_formula)
es_import = proportion_effectsize(prob_1, prob_2)
es_import

-0.02099570652984345

In [6]:
#Расчёт минимально необходимой выборки * 2(для теста и контроля) для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)
Number_of_esperements = calc_sample_size_proportion(effect_size=es_import, alpha=.05, beta=.2)
Number_of_esperements


71220

In [7]:
#Расчет количества дней для эксперемента.
Person_per_day = sample_1_size_control/43200     #43200 это количество минут в месяце


Number_of_days = Number_of_esperements / Person_per_day
Number_of_days

9.049129411764707

**Вывод:** После проведения эксперемента выявлено, что для проведения эксперемента потребуется 9 минут.Атакже принимаем тестовую версию, птому что p-value < alpha и довертельный интервал отрицательный и не прохоит через ноль.